<a href="https://colab.research.google.com/github/Zhijing-W/Transformer-in-spatial-temporal-combination-model/blob/main/rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain-community
!pip install langchain-huggingface

In [ ]:
from langchain_community.retrievers import BM25Retriever
from typing import List
import jieba
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
loader = TextLoader('/content/drive/MyDrive/Thesis Code/Huatuo_data.txt')
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 0,
    length_function = len,
    separators=['\n']
)
docs = text_splitter.split_documents(documents)

In [ ]:
import pickle

# 指定保存路径
save_path = '/content/drive/MyDrive/Thesis Code/docs.pkl'

# 保存 docs 到文件中
#with open(save_path, 'wb') as f:
#    pickle.dump(docs, f)

In [ ]:
import pickle
# 当你需要加载 docs 时，可以直接从该路径读取
with open('/content/drive/MyDrive/Thesis Code/docs.pkl', 'rb') as f:
    docs = pickle.load(f)

In [ ]:
#docs[0]

In [ ]:
pip install rank_bm25

In [ ]:
def preprocessing_func(text: str) -> List[str]:
    return list(jieba.cut(text))
bm25 = BM25Retriever(docs=docs,k=10)
print(bm25.k)
retriever = bm25.from_documents(docs,preprocess_func=preprocessing_func)

10


Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.cache
Loading model cost 0.859 seconds.
DEBUG:jieba:Loading model cost 0.859 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [ ]:
# 定义 question 变量
question = "鼻子不通气怎么解决？"

In [ ]:
retriever.invoke(question)

[Document(metadata={'source': '/content/drive/MyDrive/Thesis Code/Huatuo_data.txt'}, page_content='{"question": "一般在感冒的期间为什么会出现鼻子不通气的情况，鼻子不通气是属于什么疾病？问题和原因呢？一般用什么样的方式可以缓解？在感冒期间的鼻子不通气的情况呢？", "answer": "感冒期间鼻子不通气的原因可能是过敏性鼻炎或者是伤风所引起的，这些疾病会导致鼻腔内的黏膜肿胀，从而导致鼻子不通气。同时，感冒期间鼻腔内的黏液分泌增多，也会导致鼻子不通气。鼻子不通气是一种症状，而不是一种疾病。缓解鼻子不通气的方法包括多给鼻子增加湿气，可以用蒸汽浴或者是盐水漱口来缓解，同时也可以使用鼻子喷雾剂来缓解症状。在感冒期间，要注意保暖，避免着凉，多休息，多喝水，有助于缓解鼻子不通气的症状。"}'),
 Document(metadata={'source': '/content/drive/MyDrive/Thesis Code/Huatuo_data.txt'}, page_content='{"question": "宝宝二十个月了睡觉前总是要吵上一阵子，又不缺钙，白天睡的也不久，也不知道是怎么回事。从小吵到大，没安安稳稳的睡过一个踏实的觉。求解决", "answer": "宝宝睡觉前总是要吵上一阵子，又不缺钙，白天睡的也不久，需要考虑其他原因引起的。建议去医院检查，排除其他疾病，确诊之后制定治疗方案。"}\n{"question": "宝宝感冒鼻子不通怎么办？你好，我宝宝三岁了，感冒有三四天了今提比较厉害，睡不好觉，鼻子不通气，有没有什办法可以快点通气啊，急死人了", "answer": "根据你的描述，宝宝感冒已经持续了三四天，鼻子不通气，睡眠质量不好。如果只是鼻子不通气，没有发烧，你可以尝试使用生理盐水滴鼻。生理盐水可以帮助清洁鼻腔，缓解鼻塞的症状。如果宝宝有发烧，建议去医院就诊。发烧可能是感染的症状，需要医生的进一步诊断和治疗。希望这些信息对你有帮助，祝宝宝早日康复。"}'),
 Document(metadata={'source': '/content/drive/MyDrive/Thesis Code/Huatuo_data.txt'}, page_c

In [ ]:
from rank_bm25 import BM25Okapi
texts = [i.page_content for i in docs]
texts_processed = [preprocessing_func(t) for t in texts]
vectorizer = BM25Okapi(texts_processed)

In [ ]:
vectorizer.get_top_n(preprocessing_func(question),texts, n=10)

['{"question": "一般在感冒的期间为什么会出现鼻子不通气的情况，鼻子不通气是属于什么疾病？问题和原因呢？一般用什么样的方式可以缓解？在感冒期间的鼻子不通气的情况呢？", "answer": "感冒期间鼻子不通气的原因可能是过敏性鼻炎或者是伤风所引起的，这些疾病会导致鼻腔内的黏膜肿胀，从而导致鼻子不通气。同时，感冒期间鼻腔内的黏液分泌增多，也会导致鼻子不通气。鼻子不通气是一种症状，而不是一种疾病。缓解鼻子不通气的方法包括多给鼻子增加湿气，可以用蒸汽浴或者是盐水漱口来缓解，同时也可以使用鼻子喷雾剂来缓解症状。在感冒期间，要注意保暖，避免着凉，多休息，多喝水，有助于缓解鼻子不通气的症状。"}',
 '{"question": "宝宝二十个月了睡觉前总是要吵上一阵子，又不缺钙，白天睡的也不久，也不知道是怎么回事。从小吵到大，没安安稳稳的睡过一个踏实的觉。求解决", "answer": "宝宝睡觉前总是要吵上一阵子，又不缺钙，白天睡的也不久，需要考虑其他原因引起的。建议去医院检查，排除其他疾病，确诊之后制定治疗方案。"}\n{"question": "宝宝感冒鼻子不通怎么办？你好，我宝宝三岁了，感冒有三四天了今提比较厉害，睡不好觉，鼻子不通气，有没有什办法可以快点通气啊，急死人了", "answer": "根据你的描述，宝宝感冒已经持续了三四天，鼻子不通气，睡眠质量不好。如果只是鼻子不通气，没有发烧，你可以尝试使用生理盐水滴鼻。生理盐水可以帮助清洁鼻腔，缓解鼻塞的症状。如果宝宝有发烧，建议去医院就诊。发烧可能是感染的症状，需要医生的进一步诊断和治疗。希望这些信息对你有帮助，祝宝宝早日康复。"}',
 '{"question": "我的宝宝现在才两个月，出现鼻子不通气的情况，宝宝老是闹人，去医院才知道是因为宝宝鼻子不通气导致的，要怎么缓解宝宝鼻子不通气情况？", "answer": "宝宝两个月，鼻道还没有完全发育好，鼻子里的异物不容易自己出来，需要家长帮助清理。不可以直接用手扣宝宝的鼻子，可以用热毛巾热敷宝宝鼻子两侧，但是不可以温度过高，不然会烫伤宝宝。热敷几次后，可以挤压宝宝的鼻子两侧，把鼻屎挤出来。平时要给宝宝多喝水。"}',
 '{"question": "我有过敏性鼻炎，每年7-9月鼻子就不通气，喷药治疗，一过了9月份就自动好了，但是今年到现在鼻子还是不

In [ ]:
!pip install chromadb

!pip install -U langchain-chroma

!pip install -U langchain-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 106.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.5 MB/s eta 0:00

In [ ]:
# 导入所需的模块
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

In [ ]:
!pip uninstall numpy
!pip install numpy==1.23.5


Found existing installation: numpy 2.2.4
Uninstalling numpy-2.2.4:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/numpy-config
    /usr/local/lib/python3.11/dist-packages/numpy-2.2.4.dist-info/*
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libgfortran-040039e1-0352e75f.so.5.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libquadmath-96973f99-934c22de.so.0.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libscipy_openblas64_-6bb31eeb.so
    /usr/local/lib/python3.11/dist-packages/numpy/*
Proceed (Y/n)? Y
  Successfully uninstalled numpy-2.2.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 79.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.3.20 requires numpy<3,>=1.26.2, but you have numpy 1.23.5 which is incompatible.
chex 0.1.89 requires numpy>=1.24.1, but you have num

In [ ]:
!pip install numpy==1.23.5

In [ ]:
!pip install --upgrade pip
!pip install --upgrade numpy transformers langchain huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 68.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 144.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 132.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.0
    Uninstalling transformers-4.50.0:
      Successfully uninstalled transformers-4.50.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-chroma 0.2.2 requires numpy<2.0.0,>=1.22.4; python_version < "3.12", but you have numpy 2.2.4 which is in

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-zh-v1.5",
    model_kwargs={"device": "cuda:0"}
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/30.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.30G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
from langchain_chroma import Chroma  # 或者你使用的 Chroma 包路径
# 假设你已经用 HuggingFaceEmbeddings 加载好了 embeddings

db = Chroma(
    persist_directory="/content/drive/MyDrive/Thesis Code",
    embedding_function=embeddings
)

In [ ]:
bm25_res = vectorizer.get_top_n(preprocessing_func(question),texts, n=10)
bm25_res

['{"question": "一般在感冒的期间为什么会出现鼻子不通气的情况，鼻子不通气是属于什么疾病？问题和原因呢？一般用什么样的方式可以缓解？在感冒期间的鼻子不通气的情况呢？", "answer": "感冒期间鼻子不通气的原因可能是过敏性鼻炎或者是伤风所引起的，这些疾病会导致鼻腔内的黏膜肿胀，从而导致鼻子不通气。同时，感冒期间鼻腔内的黏液分泌增多，也会导致鼻子不通气。鼻子不通气是一种症状，而不是一种疾病。缓解鼻子不通气的方法包括多给鼻子增加湿气，可以用蒸汽浴或者是盐水漱口来缓解，同时也可以使用鼻子喷雾剂来缓解症状。在感冒期间，要注意保暖，避免着凉，多休息，多喝水，有助于缓解鼻子不通气的症状。"}',
 '{"question": "宝宝二十个月了睡觉前总是要吵上一阵子，又不缺钙，白天睡的也不久，也不知道是怎么回事。从小吵到大，没安安稳稳的睡过一个踏实的觉。求解决", "answer": "宝宝睡觉前总是要吵上一阵子，又不缺钙，白天睡的也不久，需要考虑其他原因引起的。建议去医院检查，排除其他疾病，确诊之后制定治疗方案。"}\n{"question": "宝宝感冒鼻子不通怎么办？你好，我宝宝三岁了，感冒有三四天了今提比较厉害，睡不好觉，鼻子不通气，有没有什办法可以快点通气啊，急死人了", "answer": "根据你的描述，宝宝感冒已经持续了三四天，鼻子不通气，睡眠质量不好。如果只是鼻子不通气，没有发烧，你可以尝试使用生理盐水滴鼻。生理盐水可以帮助清洁鼻腔，缓解鼻塞的症状。如果宝宝有发烧，建议去医院就诊。发烧可能是感染的症状，需要医生的进一步诊断和治疗。希望这些信息对你有帮助，祝宝宝早日康复。"}',
 '{"question": "我的宝宝现在才两个月，出现鼻子不通气的情况，宝宝老是闹人，去医院才知道是因为宝宝鼻子不通气导致的，要怎么缓解宝宝鼻子不通气情况？", "answer": "宝宝两个月，鼻道还没有完全发育好，鼻子里的异物不容易自己出来，需要家长帮助清理。不可以直接用手扣宝宝的鼻子，可以用热毛巾热敷宝宝鼻子两侧，但是不可以温度过高，不然会烫伤宝宝。热敷几次后，可以挤压宝宝的鼻子两侧，把鼻屎挤出来。平时要给宝宝多喝水。"}',
 '{"question": "我有过敏性鼻炎，每年7-9月鼻子就不通气，喷药治疗，一过了9月份就自动好了，但是今年到现在鼻子还是不

In [ ]:
vector_res = db.similarity_search(question, k=10)
vector_res

[Document(id='a9a50bf2-7feb-4a5e-86a7-cf710a101a39', metadata={'source': '/content/drive/MyDrive/Thesis Code/Huatuo_data.txt'}, page_content='{"question": "我一直有慢性鼻炎，这不一到天气变化，鼻炎复发了。现在两个鼻子鼻塞的厉害，只能用嘴巴来呼吸了，平时睡觉时更难受，经常被自己憋醒。我吃过很多治疗鼻塞得药了，但是效果不明显。有治疗鼻子不通气小妙招吗？", "answer": "鼻子不通气的小妙招有很多，你可以尝试以下方法：\\n1. 使用鼻腔粘膜收缩剂滴鼻治疗，可以缓解鼻腔通气。\\n2. 考虑手术治疗，切除部分鼻甲缓解通气。\\n3. 凉水洗鼻腔，可以缓解鼻塞。\\n4. 将大蒜塞进鼻子里，可以尝试一下。\\n5. 吸入水蒸气，可以用口鼻对准盛有开水的杯子，深呼气、深吸气，每次持续20分钟左右。\\n6. 尝试将葱白或洋葱切碎煎汤，用鼻吸热气，或将食醋烧开吸醋气，疗效都较好。\\n\\n但是请注意，每个人的情况可能不同，效果也会有所差异。如果这些方法不起作用，建议你咨询医生，了解更适合你的治疗方法。"}'),
 Document(id='e4a20591-0aa7-442d-8a26-3c548b48f33d', metadata={'source': '/content/drive/MyDrive/Thesis Code/Huatuo_data.txt'}, page_content='{"question": "很小的时候一感冒鼻子就不通气，吃过匹多莫德颗粒，原来挺管事的，后来再吃就不管用了，现在天气冷了，孩子即使没有感冒症状鼻子也不通气，晚上睡觉打呼噜很难受的。到底怎样让孩子鼻子能通气啊？", "answer": "孩子感冒后鼻子不通气可能有多种原因。一种可能是鼻子的黏膜受到感染或炎症，导致鼻腔堵塞。另一种可能是孩子的鼻子过敏，对某些物质过敏反应导致鼻塞。此外，气候变化、干燥的室内空气、空气污染等也可能影响孩子的鼻子通气。\\n\\n为了改善孩子的鼻子通气问题，你可以尝试以下方法：\\n1. 保持室内空气湿润：使用加湿器或放置水盆来增加室内湿度，这有助于减少鼻腔干燥和堵塞。\\n2. 温水洗鼻：使用温水和盐

In [ ]:
def rrf(vector_results: List[str], text_results: List[str], k: int=10, m: int=60):
        """
        使用RRF算法对两组检索结果进行重排序

        params:
        vector_results (list): 向量召回的结果列表,每个元素是专利ID
        text_results (list): 文本召回的结果列表,每个元素是专利ID
        k(int): 排序后返回前k个
        m (int): 超参数

        return:
        重排序后的结果列表,每个元素是(文档ID, 融合分数)
        """

        doc_scores = {}

        # 遍历两组结果,计算每个文档的融合分数
        for rank, doc_id in enumerate(vector_results):
            doc_scores[doc_id] = doc_scores.get(doc_id, 0) + 1 / (rank+m)
        for rank, doc_id in enumerate(text_results):
            doc_scores[doc_id] = doc_scores.get(doc_id, 0) + 1 / (rank+m)

        # 将结果按融合分数排序
        sorted_results = [d for d, _ in sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)[:k]]

        return sorted_results

In [ ]:
vector_results = [i.page_content for i in vector_res]
text_results = [i for i in bm25_res]
rrf_res = rrf(vector_results, text_results)
rrf_res

['{"question": "我一直有慢性鼻炎，这不一到天气变化，鼻炎复发了。现在两个鼻子鼻塞的厉害，只能用嘴巴来呼吸了，平时睡觉时更难受，经常被自己憋醒。我吃过很多治疗鼻塞得药了，但是效果不明显。有治疗鼻子不通气小妙招吗？", "answer": "鼻子不通气的小妙招有很多，你可以尝试以下方法：\\n1. 使用鼻腔粘膜收缩剂滴鼻治疗，可以缓解鼻腔通气。\\n2. 考虑手术治疗，切除部分鼻甲缓解通气。\\n3. 凉水洗鼻腔，可以缓解鼻塞。\\n4. 将大蒜塞进鼻子里，可以尝试一下。\\n5. 吸入水蒸气，可以用口鼻对准盛有开水的杯子，深呼气、深吸气，每次持续20分钟左右。\\n6. 尝试将葱白或洋葱切碎煎汤，用鼻吸热气，或将食醋烧开吸醋气，疗效都较好。\\n\\n但是请注意，每个人的情况可能不同，效果也会有所差异。如果这些方法不起作用，建议你咨询医生，了解更适合你的治疗方法。"}',
 '{"question": "一般在感冒的期间为什么会出现鼻子不通气的情况，鼻子不通气是属于什么疾病？问题和原因呢？一般用什么样的方式可以缓解？在感冒期间的鼻子不通气的情况呢？", "answer": "感冒期间鼻子不通气的原因可能是过敏性鼻炎或者是伤风所引起的，这些疾病会导致鼻腔内的黏膜肿胀，从而导致鼻子不通气。同时，感冒期间鼻腔内的黏液分泌增多，也会导致鼻子不通气。鼻子不通气是一种症状，而不是一种疾病。缓解鼻子不通气的方法包括多给鼻子增加湿气，可以用蒸汽浴或者是盐水漱口来缓解，同时也可以使用鼻子喷雾剂来缓解症状。在感冒期间，要注意保暖，避免着凉，多休息，多喝水，有助于缓解鼻子不通气的症状。"}',
 '{"question": "很小的时候一感冒鼻子就不通气，吃过匹多莫德颗粒，原来挺管事的，后来再吃就不管用了，现在天气冷了，孩子即使没有感冒症状鼻子也不通气，晚上睡觉打呼噜很难受的。到底怎样让孩子鼻子能通气啊？", "answer": "孩子感冒后鼻子不通气可能有多种原因。一种可能是鼻子的黏膜受到感染或炎症，导致鼻腔堵塞。另一种可能是孩子的鼻子过敏，对某些物质过敏反应导致鼻塞。此外，气候变化、干燥的室内空气、空气污染等也可能影响孩子的鼻子通气。\\n\\n为了改善孩子的鼻子通气问题，你可以尝试以下方法：\\n1. 保持室内空气湿润：使用加湿器或放置水盆来增加室内湿度，这有助于减少

In [ ]:
!pip install langchain_openai

from langchain_openai import ChatOpenAI

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.9 MB/s eta 0:00:00


In [ ]:
rrf_res=['{"question": "我一直有慢性鼻炎，这不一到天气变化，鼻炎复发了。现在两个鼻子鼻塞的厉害，只能用嘴巴来呼吸了，平时睡觉时更难受，经常被自己憋醒。我吃过很多治疗鼻塞得药了，但是效果不明显。有治疗鼻子不通气小妙招吗？", "answer": "鼻子不通气的小妙招有很多，你可以尝试以下方法：\\n1. 使用鼻腔粘膜收缩剂滴鼻治疗，可以缓解鼻腔通气。\\n2. 考虑手术治疗，切除部分鼻甲缓解通气。\\n3. 凉水洗鼻腔，可以缓解鼻塞。\\n4. 将大蒜塞进鼻子里，可以尝试一下。\\n5. 吸入水蒸气，可以用口鼻对准盛有开水的杯子，深呼气、深吸气，每次持续20分钟左右。\\n6. 尝试将葱白或洋葱切碎煎汤，用鼻吸热气，或将食醋烧开吸醋气，疗效都较好。\\n\\n但是请注意，每个人的情况可能不同，效果也会有所差异。如果这些方法不起作用，建议你咨询医生，了解更适合你的治疗方法。"}',
 '{"question": "一般在感冒的期间为什么会出现鼻子不通气的情况，鼻子不通气是属于什么疾病？问题和原因呢？一般用什么样的方式可以缓解？在感冒期间的鼻子不通气的情况呢？", "answer": "感冒期间鼻子不通气的原因可能是过敏性鼻炎或者是伤风所引起的，这些疾病会导致鼻腔内的黏膜肿胀，从而导致鼻子不通气。同时，感冒期间鼻腔内的黏液分泌增多，也会导致鼻子不通气。鼻子不通气是一种症状，而不是一种疾病。缓解鼻子不通气的方法包括多给鼻子增加湿气，可以用蒸汽浴或者是盐水漱口来缓解，同时也可以使用鼻子喷雾剂来缓解症状。在感冒期间，要注意保暖，避免着凉，多休息，多喝水，有助于缓解鼻子不通气的症状。"}',
 '{"question": "很小的时候一感冒鼻子就不通气，吃过匹多莫德颗粒，原来挺管事的，后来再吃就不管用了，现在天气冷了，孩子即使没有感冒症状鼻子也不通气，晚上睡觉打呼噜很难受的。到底怎样让孩子鼻子能通气啊？", "answer": "孩子感冒后鼻子不通气可能有多种原因。一种可能是鼻子的黏膜受到感染或炎症，导致鼻腔堵塞。另一种可能是孩子的鼻子过敏，对某些物质过敏反应导致鼻塞。此外，气候变化、干燥的室内空气、空气污染等也可能影响孩子的鼻子通气。\\n\\n为了改善孩子的鼻子通气问题，你可以尝试以下方法：\\n1. 保持室内空气湿润：使用加湿器或放置水盆来增加室内湿度，这有助于减少鼻腔干燥和堵塞。\\n2. 温水洗鼻：使用温水和盐水混合物进行鼻腔冲洗，可以清洁鼻腔，减少鼻塞。\\n3. 避免过敏原：尽量避免孩子接触可能引起过敏的物质，如花粉、灰尘、宠物毛发等。\\n4. 饮食调理：给孩子提供均衡的饮食，多吃新鲜蔬菜和水果，增强免疫力。\\n5. 适当锻炼：适当的体育锻炼可以增强孩子的免疫力，有助于改善鼻子通气问题。\\n\\n如果以上方法无效，建议咨询儿科医生，以获取更专业的建议和治疗方案。"}',
 '{"question": "宝宝二十个月了睡觉前总是要吵上一阵子，又不缺钙，白天睡的也不久，也不知道是怎么回事。从小吵到大，没安安稳稳的睡过一个踏实的觉。求解决", "answer": "宝宝睡觉前总是要吵上一阵子，又不缺钙，白天睡的也不久，需要考虑其他原因引起的。建议去医院检查，排除其他疾病，确诊之后制定治疗方案。"}\n{"question": "宝宝感冒鼻子不通怎么办？你好，我宝宝三岁了，感冒有三四天了今提比较厉害，睡不好觉，鼻子不通气，有没有什办法可以快点通气啊，急死人了", "answer": "根据你的描述，宝宝感冒已经持续了三四天，鼻子不通气，睡眠质量不好。如果只是鼻子不通气，没有发烧，你可以尝试使用生理盐水滴鼻。生理盐水可以帮助清洁鼻腔，缓解鼻塞的症状。如果宝宝有发烧，建议去医院就诊。发烧可能是感染的症状，需要医生的进一步诊断和治疗。希望这些信息对你有帮助，祝宝宝早日康复。"}',
 '{"question": "去年11月份左右感冒好了之后开始鼻子不晓得怎么了就一直不通，现在基本每天要洗鼻子1次，用过鼻立爽喷剂，用了有34个月的时间，效果不理想，之后又在网上买了一个中药的滴液，虽然使用后鼻通时间能维持在67个小时，但没有改善效果，用了差不多5个月的时间，还有就是我一闻到刺鼻、香烟什么的，鼻子就开始痒，打喷嚏、流鼻涕。:需要医生帮助提供远程诊断：请问我这是过敏性鼻炎嘛，有什么方法能让鼻通，长期鼻子不通很烦恼。", "answer": "您好！根据您的描述，您可能患有过敏性鼻炎。建议您尽快咨询医生并进行鼻内窥镜检查，以确认诊断并得到针对性的治疗。除此之外，您可以每天用淡盐水清洗鼻腔，这是一种可以缓解鼻塞的家庭疗法。希望这对您有帮助！如有更多问题，请随时追问。"}',
 '{"question": "我的宝宝现在才两个月，出现鼻子不通气的情况，宝宝老是闹人，去医院才知道是因为宝宝鼻子不通气导致的，要怎么缓解宝宝鼻子不通气情况？", "answer": "宝宝两个月，鼻道还没有完全发育好，鼻子里的异物不容易自己出来，需要家长帮助清理。不可以直接用手扣宝宝的鼻子，可以用热毛巾热敷宝宝鼻子两侧，但是不可以温度过高，不然会烫伤宝宝。热敷几次后，可以挤压宝宝的鼻子两侧，把鼻屎挤出来。平时要给宝宝多喝水。"}',
 '{"question": "鼻子不通经常感冒总感觉鼻子不是很通，平时还好，但总是很容易感冒，每次一感冒，鼻子很难受，堵得慌，而且平时感觉嗅觉不是很灵敏，轻微的味道周围的朋友能闻到，但是我感觉不出来，已经这样几年了，想咨询一下是什么情况？以及相关治疗方法谢谢", "answer": "根据症状描述，这位用户可能患有鼻炎，鼻炎是一种常见的疾病，多数情况下是由感冒引起的，症状包括鼻塞、流涕、头痛等。鼻炎的治疗方法包括药物治疗和非药物治疗，药物治疗包括口服和局部用药，非药物治疗包括鼻腔冲洗、鼻腔按摩等。建议用户及时就医，根据医生的建议进行治疗。"}',
 '{"question": "我有过敏性鼻炎，每年7-9月鼻子就不通气，喷药治疗，一过了9月份就自动好了，但是今年到现在鼻子还是不通气，怎么回事？。病情描述（发病时间、主要症状等）：喷药治疗怎么治疗，才能好，不要喷药", "answer": "根据病情描述，患者有过敏性鼻炎，每年7-9月鼻子不通气，喷药治疗，一过了9月份就自动好了，但是今年到现在鼻子还是不通气。可能是患者的过敏原发生了变化，导致治疗效果不佳。建议患者去医院进行过敏原检测，找到过敏原并避免接触，同时可以使用扑尔敏、强的松等药物进行治疗。"}',
 '{"question": "最近重感冒，鼻子也堵塞的厉害，晚上睡觉非常的难受，都睡不好，呼吸困难的，影响到我白天时候的工作，有时候方法可以治疗鼻子不通吗？用点什么药好的快？", "answer": "最近重感冒，鼻子也堵塞的厉害，晚上睡觉非常的难受，都睡不好，呼吸困难的，影响到我白天时候的工作，有时候方法可以治疗鼻子不通吗？用点什么药好的快？\\n\\n参考答案提供了一种传统的方法来缓解感冒引起的鼻塞。它建议用手掌对搓产生热量，然后按摩迎香穴（位于鼻沟内、横平鼻外缘中点）约十次。这种方法可以帮助预防感冒，并在感冒后减轻鼻塞症状。此外，它建议在睡觉前将一根新鲜的葱条插入每个鼻孔，3小时后取出。这种方法通常一次就可以缓解鼻塞。但如果需要，可以在第二天再次尝试。\\n\\n需要注意的是，这些方法可能不适合每个人。因此，最好咨询医疗专业人士以获得个性化建议，并确定是否需要任何药物治疗。"}',
 '{"question": "肠癌的人会不会长期便血而不存在其他症状的可能，还是一旦出现症状就会多种症状并存，比如不仅便血而且腹痛等", "answer": "肠癌的人有可能长期出现便血而不存在其他症状。然而，一旦出现症状，通常会有多种症状并存，比如腹痛等。建议患者到正规医院就诊，寻求中医的治疗方法，同时注意饮食，避免辛辣刺激性食物。"}\n{"question": "从前天开始就出现了感冒的症状，我一直担心会变得严重，所以想了解一下宝宝感冒了鼻子不通气怎么办才好呢？", "answer": "宝宝感冒了鼻子不通气要注意及时性的药物治疗。小孩子感冒发生鼻子不通气的情况，可以使用通气鼻贴进行缓解，也可以给孩子做好保暖措施，发汗有助于缓解，鼻子不通气，吃一些有助于发暖的食物，也有缓解症状的作用，另外，让孩子保持鼻腔内清洁对于改善鼻子不通气也有很好的作用。"}']

In [ ]:
prompt = '''
任务目标：仔细分析检索出的文档即rrf_res，并结合rrf_res对应的一些列answer，对模型的输出进行改进
任务要求：
    1、仔细分析检索出的文档即rrf_res
    2、显示rrf_res中对应的answer数量，以及你决定要参考的一系列answer，以及其对应的数量
    3、若检索出的文档不包含用户问题的答案，请回答我不知道

用户问题：
{}

检索出的文档：
{}
'''

In [ ]:
# 构造你的 prompt，注意这里需要根据你实际的 prompt 格式进行定义
prompt_text = prompt.format(question, ''.join(rrf_res))

In [ ]:
api_address = "https://api.siliconflow.cn"
api_key = "sk-ukjwcaqerxqefiidtctmdbspbsdnxlateyytobbuterjsfbb"

# 根据实际情况选择模型名称，此处以 deepseek-r1 为例
model = ChatOpenAI(model='deepseek-ai/DeepSeek-R1-Distill-Qwen-14B', base_url=api_address, api_key=api_key)


In [ ]:
api_address = "https://api.siliconflow.cn"
api_key = "sk-ukjwcaqerxqefiidtctmdbspbsdnxlateyytobbuterjsfbb"

# 根据实际情况选择模型名称，此处以 deepseek-r1 为例
model = ChatOpenAI(model='Qwen/QwQ-32B', base_url=api_address, api_key=api_key)

In [ ]:
rag_answer = model.invoke(prompt.format(prompt_text, ''.join(rrf_res)))
print(rag_answer.content)



### 分析与改进后的回答：

#### 1. **检索文档分析**：
- **总数**：共 **11 条文档**，涉及成人和儿童鼻塞的不同原因（感冒、过敏性鼻炎、慢性鼻炎、鼻甲肥大等）及解决方案。
- **关键信息提取**：
  - **高频推荐方法**：生理盐水冲洗、蒸汽吸入、保持湿度、避免过敏原、咨询医生。
  - **需谨慎的方法**：大蒜/葱条塞鼻可能刺激黏膜，2个月宝宝需用安全物理法（热敷、挤压鼻屎）。
  - **医学建议**：反复无效或伴随发烧需就医，明确病因如过敏性鼻炎或鼻甲问题。

---

#### 2. **参考答案统计**：
- **答案数量**：共有 **11 个 answer**，其中针对“鼻子不通气解决方案”的建议如下（按推荐频率排序）：
  1. **生理盐水冲洗/滴鼻**：提出 **4次**（适用于成人、儿童、新生儿，安全有效）。
  2. **蒸汽吸入或热敷**：提出 **3次**（注意温度避免烫伤，尤其孩子）。
  3. **保持环境湿度**：提出 **3次**（用加湿器缓解干燥鼻塞）。
  4. **避免过敏原与抗过敏管理**：提出 **4次**（如花粉、尘螨，需检测过敏原）。
  5. **药物辅助**：喷雾剂、收缩剂等 **3次**，但需遵医嘱（慢性鼻炎可能需手术）。
  6. **饮食与生活习惯**：均衡饮食、多喝水、适当运动（**2次**）。

---

#### 3. **改进后的优化回答**：
---

##### **解决鼻子不通气的办法**：
根据你的具体症状，以下方法供参考：

---

**❗ 核心推荐方案：**
1. **盐水冲洗法**（适用所有年龄）：
   - **成人/大童**：用生理盐水鼻喷或洗鼻器冲洗鼻腔，清洁分泌物并缓解肿胀。
   - **婴儿**：用棉签蘸生理盐水轻柔软化鼻痂，再用婴儿吸鼻器吸出（如2个月宝宝）。

2. **蒸汽吸入法**：
   - **成人**：将热水（30℃左右）倒入盆中，弯腰用毛巾盖头吸入5-10分钟，每天2-3次（注意防烫伤）。
   - **儿童**：可减少水量或搭配加湿器，避免直接接触高温蒸汽。

3. **环境与生活习惯调整**：
   - **加湿空气**：室内湿度保持在40%-60%，使用加湿器或放置水盆。
   - **防过敏源**：定期清洁床

In [ ]:
no_rag_answer = model.invoke(question)
print(no_rag_answer.content)



鼻子不通气（鼻塞）可能由多种原因引起，如感冒、过敏、鼻窦炎、鼻中隔偏曲、环境干燥或过度使用减充血剂等。以下是针对不同情况的应对措施，可分为家庭护理、药物辅助和需要就医的情况：

---

### **一、家庭护理方法**
#### 1. **蒸汽吸入**
   - **方法**：用热水倒入大碗中，头靠近碗口，用毛巾盖住头部形成蒸汽罩，吸入热蒸汽5-10分钟。可加入几滴薄荷或桉树精油增强效果。
   - **作用**：缓解鼻腔干燥或充血，稀释黏液。
   - **注意**：避免烫伤，保持安全距离。

#### 2. **盐水冲洗/喷雾**
   - **方法**：
     - **生理盐水冲洗**：使用洗鼻壶或喷雾器，将生理盐水（比例为1升温水+1茶匙盐，煮沸冷却后使用）冲洗鼻腔，清除分泌物和过敏原。
     - **鼻用盐水喷雾**：药店购买现成产品，可随身携带。
   - **作用**：温和清洁鼻腔，保持湿润，缓解充血。

#### 3. **热敷**
   - **方法**：用温热（非烫）的湿毛巾敷在鼻梁和鼻窦区域（额头、颧骨、太阳穴），每次10-15分钟，每日数次。
   - **作用**：促进血液循环，缓解鼻窦肿胀。

#### 4. **保持空气湿润**
   - **方法**：使用加湿器或放置水盆在房间内，增加空气湿度（建议湿度在40%-60%），避免鼻腔干燥。
   - **注意**：定期清洁加湿器，防止霉菌滋生。

#### 5. **头部抬高**
   - **方法**：睡觉时用额外枕头将头和上半身抬高15-30度，或调整睡姿，减少鼻腔液体倒流引起的鼻塞。

---

### **二、药物缓解（按需使用）**
#### 1. **临时解充血喷雾**
   - **成分**：羟甲唑啉（Afrin）、赛洛唑啉等。
   - **使用方法**：遵医嘱或说明书，连续使用不超过3天（避免药物性鼻炎）。
   - **注意**：孕妇、高血压患者需谨慎。

#### 2. **抗组胺药（过敏性鼻塞）**
   - **适用情况**：花粉、尘螨、宠物皮屑等过敏引起鼻塞。
   - **药物举例**：西替利嗪、氯雷他定、非索非那定（口服）；氮卓斯汀鼻喷雾（局部）。
   - **注意**：可能引起嗜睡，选择非镇静类成分。

#### 3. **抗炎鼻喷剂（鼻

In [ ]:
pip install nltk rouge-score bert-score jieba


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.1 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=2

In [ ]:
import nltk
import jieba
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.gleu_score import sentence_gleu
from rouge_score import rouge_scorer
import bert_score

# 中文分词函数（使用 jieba）
def tokenize_zh(text):
    return list(jieba.cut(text))

# 计算 BLEU_n（采用对每个参考分别计算再取平均的方式）
def compute_bleu_n(references, candidate, weights):
    ref_tokens = [tokenize_zh(ref) for ref in references]
    cand_tokens = tokenize_zh(candidate)
    scores = [sentence_bleu([ref], cand_tokens, weights=weights) for ref in ref_tokens]
    return sum(scores) / len(scores)

# 计算 GLEU（对每个参考分别计算再取平均）
def compute_gleu(references, candidate):
    ref_tokens = [tokenize_zh(ref) for ref in references]
    cand_tokens = tokenize_zh(candidate)
    scores = [sentence_gleu([ref], cand_tokens) for ref in ref_tokens]
    return sum(scores) / len(scores)

# 计算 ROUGE 各指标：ROUGE-1, ROUGE-2, ROUGE-L（取多个参考的平均值）
def compute_rouge_scores(references, candidate):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)
    rouge1_scores, rouge2_scores, rougeL_scores = [], [], []
    for ref in references:
        scores = scorer.score(ref, candidate)
        rouge1_scores.append(scores['rouge1'].fmeasure)
        rouge2_scores.append(scores['rouge2'].fmeasure)
        rougeL_scores.append(scores['rougeL'].fmeasure)
    return (sum(rouge1_scores) / len(rouge1_scores),
            sum(rouge2_scores) / len(rouge2_scores),
            sum(rougeL_scores) / len(rougeL_scores))

# 计算 BERTScore（F1 值）
def compute_bert_score(references, candidate):
    # 构造与参考数量相同的 candidate 列表
    P, R, F1 = bert_score.score([candidate] * len(references), references, lang='zh', verbose=False)
    return F1.mean().item()

# 计算 Distinct 指标（n-gram 级别）
def compute_distinct(candidate, n):
    tokens = tokenize_zh(candidate)
    if len(tokens) < n:
        return 0.0
    ngrams = [tuple(tokens[i:i+n]) for i in range(len(tokens) - n + 1)]
    return len(set(ngrams)) / len(ngrams)

# 示例数据：参考答案（多个参考答案拼接而成）
reference_answers = [
    '{"question": "我一直有慢性鼻炎，这不一到天气变化，鼻炎复发了。现在两个鼻子鼻塞的厉害，只能用嘴巴来呼吸了，平时睡觉时更难受，经常被自己憋醒。我吃过很多治疗鼻塞得药了，但是效果不明显。有治疗鼻子不通气小妙招吗？", "answer": "鼻子不通气的小妙招有很多，你可以尝试以下方法：\\n1. 使用鼻腔粘膜收缩剂滴鼻治疗，可以缓解鼻腔通气。\\n2. 考虑手术治疗，切除部分鼻甲缓解通气。\\n3. 凉水洗鼻腔，可以缓解鼻塞。\\n4. 将大蒜塞进鼻子里，可以尝试一下。\\n5. 吸入水蒸气，可以用口鼻对准盛有开水的杯子，深呼气、深吸气，每次持续20分钟左右。\\n6. 尝试将葱白或洋葱切碎煎汤，用鼻吸热气，或将食醋烧开吸醋气，疗效都较好。\\n\\n但是请注意，每个人的情况可能不同，效果也会有所差异。如果这些方法不起作用，建议你咨询医生，了解更适合你的治疗方法。"}',
    '{"question": "一般在感冒的期间为什么会出现鼻子不通气的情况，鼻子不通气是属于什么疾病？问题和原因呢？一般用什么样的方式可以缓解？在感冒期间的鼻子不通气的情况呢？", "answer": "感冒期间鼻子不通气的原因可能是过敏性鼻炎或者是伤风所引起的，这些疾病会导致鼻腔内的黏膜肿胀，从而导致鼻子不通气。同时，感冒期间鼻腔内的黏液分泌增多，也会导致鼻子不通气。鼻子不通气是一种症状，而不是一种疾病。缓解鼻子不通气的方法包括多给鼻子增加湿气，可以用蒸汽浴或者是盐水漱口来缓解，同时也可以使用鼻子喷雾剂来缓解症状。在感冒期间，要注意保暖，避免着凉，多休息，多喝水，有助于缓解鼻子不通气的症状。"}',
    '{"question": "很小的时候一感冒鼻子就不通气，吃过匹多莫德颗粒，原来挺管事的，后来再吃就不管用了，现在天气冷了，孩子即使没有感冒症状鼻子也不通气，晚上睡觉打呼噜很难受的。到底怎样让孩子鼻子能通气啊？", "answer": "孩子感冒后鼻子不通气可能有多种原因。一种可能是鼻子的黏膜受到感染或炎症，导致鼻腔堵塞。另一种可能是孩子的鼻子过敏，对某些物质过敏反应导致鼻塞。此外，气候变化、干燥的室内空气、空气污染等也可能影响孩子的鼻子通气。\\n\\n为了改善孩子的鼻子通气问题，你可以尝试以下方法：\\n1. 保持室内空气湿润：使用加湿器或放置水盆来增加室内湿度，这有助于减少鼻腔干燥和堵塞。\\n2. 温水洗鼻：使用温水和盐水混合物进行鼻腔冲洗，可以清洁鼻腔，减少鼻塞。\\n3. 避免过敏原：尽量避免孩子接触可能引起过敏的物质，如花粉、灰尘、宠物毛发等。\\n4. 饮食调理：给孩子提供均衡的饮食，多吃新鲜蔬菜和水果，增强免疫力。\\n5. 适当锻炼：适当的体育锻炼可以增强孩子的免疫力，有助于改善鼻子通气问题。\\n\\n如果以上方法无效，建议咨询儿科医生，以获取更专业的建议和治疗方案。"}',
    '{"question": "宝宝二十个月了睡觉前总是要吵上一阵子，又不缺钙，白天睡的也不久，也不知道是怎么回事。从小吵到大，没安安稳稳的睡过一个踏实的觉。求解决", "answer": "宝宝睡觉前总是要吵上一阵子，又不缺钙，白天睡的也不久，需要考虑其他原因引起的。建议去医院检查，排除其他疾病，确诊之后制定治疗方案。"}',
    '{"question": "去年11月份左右感冒好了之后开始鼻子不晓得怎么了就一直不通，现在基本每天要洗鼻子1次，用过鼻立爽喷剂，用了有34个月的时间，效果不理想，之后又在网上买了一个中药的滴液，虽然使用后鼻通时间能维持在67个小时，但没有改善效果，用了差不多5个月的时间，还有就是我一闻到刺鼻、香烟什么的，鼻子就开始痒，打喷嚏、流鼻涕。:需要医生帮助提供远程诊断：请问我这是过敏性鼻炎嘛，有什么方法能让鼻通，长期鼻子不通很烦恼。", "answer": "您好！根据您的描述，您可能患有过敏性鼻炎。建议您尽快咨询医生并进行鼻内窥镜检查，以确认诊断并得到针对性的治疗。除此之外，您可以每天用淡盐水清洗鼻腔，这是一种可以缓解鼻塞的家庭疗法。希望这对您有帮助！如有更多问题，请随时追问。"}',
    '{"question": "我的宝宝现在才两个月，出现鼻子不通气的情况，宝宝老是闹人，去医院才知道是因为宝宝鼻子不通气导致的，要怎么缓解宝宝鼻子不通气情况？", "answer": "宝宝两个月，鼻道还没有完全发育好，鼻子里的异物不容易自己出来，需要家长帮助清理。不可以直接用手扣宝宝的鼻子，可以用热毛巾热敷宝宝鼻子两侧，但是不可以温度过高，不然会烫伤宝宝。热敷几次后，可以挤压宝宝的鼻子两侧，把鼻屎挤出来。平时要给宝宝多喝水。"}',
    '{"question": "鼻子不通经常感冒总感觉鼻子不是很通，平时还好，但总是很容易感冒，每次一感冒，鼻子很难受，堵得慌，而且平时感觉嗅觉不是很灵敏，轻微的味道周围的朋友能闻到，但是我感觉不出来，已经这样几年了，想咨询一下是什么情况？以及相关治疗方法谢谢", "answer": "根据症状描述，这位用户可能患有鼻炎，鼻炎是一种常见的疾病，多数情况下是由感冒引起的，症状包括鼻塞、流涕、头痛等。鼻炎的治疗方法包括药物治疗和非药物治疗，药物治疗包括口服和局部用药，非药物治疗包括鼻腔冲洗、鼻腔按摩等。建议用户及时就医，根据医生的建议进行治疗。"}',
    '{"question": "我有过敏性鼻炎，每年7-9月鼻子就不通气，喷药治疗，一过了9月份就自动好了，但是今年到现在鼻子还是不通气，怎么回事？。病情描述（发病时间、主要症状等）：喷药治疗怎么治疗，才能好，不要喷药", "answer": "根据病情描述，患者有过敏性鼻炎，每年7-9月鼻子不通气，喷药治疗，一过了9月份就自动好了，但是今年到现在鼻子还是不通气。可能是患者的过敏原发生了变化，导致治疗效果不佳。建议患者去医院进行过敏原检测，找到过敏原并避免接触，同时可以使用扑尔敏、强的松等药物进行治疗。"}',
    '{"question": "最近重感冒，鼻子也堵塞的厉害，晚上睡觉非常的难受，都睡不好，呼吸困难的，影响到我白天时候的工作，有时候方法可以治疗鼻子不通吗？用点什么药好的快？", "answer": "最近重感冒，鼻子也堵塞的厉害，晚上睡觉非常的难受，都睡不好，呼吸困难的，影响到我白天时候的工作，有时候方法可以治疗鼻子不通吗？用点什么药好的快？\\n\\n参考答案提供了一种传统的方法来缓解感冒引起的鼻塞。它建议用手掌对搓产生热量，然后按摩迎香穴（位于鼻沟内、横平鼻外缘中点）约十次。这种方法可以帮助预防感冒，并在感冒后减轻鼻塞症状。此外，它建议在睡觉前将一根新鲜的葱条插入每个鼻孔，3小时后取出。这种方法通常一次就可以缓解鼻塞。但如果需要，可以在第二天再次尝试。\\n\\n需要注意的是，这些方法可能不适合每个人。因此，最好咨询医疗专业人士以获得个性化建议，并确定是否需要任何药物治疗。"}',
    '{"question": "肠癌的人会不会长期便血而不存在其他症状的可能，还是一旦出现症状就会多种症状并存，比如不仅便血而且腹痛等", "answer": "肠癌的人有可能长期出现便血而不存在其他症状。然而，一旦出现症状，通常会有多种症状并存，比如腹痛等。建议患者到正规医院就诊，寻求中医的治疗方法，同时注意饮食，避免辛辣刺激性食物。"}',
    '{"question": "从前天开始就出现了感冒的症状，我一直担心会变得严重，所以想了解一下宝宝感冒了鼻子不通气怎么办才好呢？", "answer": "宝宝感冒了鼻子不通气要注意及时性的药物治疗。小孩子感冒发生鼻子不通气的情况，可以使用通气鼻贴进行缓解，也可以给孩子做好保暖措施，发汗有助于缓解，鼻子不通气，吃一些有助于发暖的食物，也有缓解症状的作用，另外，让孩子保持鼻腔内清洁对于改善鼻子不通气也有很好的作用。"}'
]

# 两种候选答案（例如：使用了 RAG 与未使用 RAG 的结果）
answer_with_rag = """分析与改进后的回答：
1. 检索文档分析：
•	总数：共 11 条文档，涉及成人和儿童鼻塞的不同原因（感冒、过敏性鼻炎、慢性鼻炎、鼻甲肥大等）及解决方案。
•	关键信息提取：
o	高频推荐方法：生理盐水冲洗、蒸汽吸入、保持湿度、避免过敏原、咨询医生。
o	需谨慎的方法：大蒜/葱条塞鼻可能刺激黏膜，2个月宝宝需用安全物理法（热敷、挤压鼻屎）。
o	医学建议：反复无效或伴随发烧需就医，明确病因如过敏性鼻炎或鼻甲问题。

2. 参考答案统计：
•	答案数量：共有 11 个 answer，其中针对“鼻子不通气解决方案”的建议如下（按推荐频率排序）：
a.	生理盐水冲洗/滴鼻：提出 4次（适用于成人、儿童、新生儿，安全有效）。
b.	蒸汽吸入或热敷：提出 3次（注意温度避免烫伤，尤其孩子）。
c.	保持环境湿度：提出 3次（用加湿器缓解干燥鼻塞）。
d.	避免过敏原与抗过敏管理：提出 4次（如花粉、尘螨，需检测过敏原）。
e.	药物辅助：喷雾剂、收缩剂等 3次，但需遵医嘱（慢性鼻炎可能需手术）。
f.	饮食与生活习惯：均衡饮食、多喝水、适当运动（2次）。

3. 改进后的优化回答：

解决鼻子不通气的办法：
根据你的具体症状，以下方法供参考：

❗ 核心推荐方案：
1.	盐水冲洗法（适用所有年龄）：
o	成人/大童：用生理盐水鼻喷或洗鼻器冲洗鼻腔，清洁分泌物并缓解肿胀。
o	婴儿：用棉签蘸生理盐水轻柔软化鼻痂，再用婴儿吸鼻器吸出（如2个月宝宝）。
2.	蒸汽吸入法：
o	成人：将热水（30℃左右）倒入盆中，弯腰用毛巾盖头吸入5-10分钟，每天2-3次（注意防烫伤）。
o	儿童：可减少水量或搭配加湿器，避免直接接触高温蒸汽。
3.	环境与生活习惯调整：
o	加湿空气：室内湿度保持在40%-60%，使用加湿器或放置水盆。
o	防过敏源：定期清洁床上用品，减少尘螨；避免接触烟雾、花粉等刺激物。

⚠️ 风险提醒（谨慎或需医生指导的方法）：
•	✅ 可用：葱白/洋葱煮汤蒸气吸入（但不可直接塞鼻孔）。
•	❌ 慎用：大蒜或葱条塞鼻（易刺激黏膜，尤其儿童）；鼻甲手术需专业评估。
•	⚠️ 若伴随以下情况，立即就医：
o	鼻塞持续超2周无改善。
o	发烧、流脓涕、嗅觉严重减退。
o	儿童睡眠严重受影响，呼吸急促或缺氧。

💊 药物与进阶建议：
•	短期缓解：非处方鼻用减充血剂（如伪麻黄碱喷雾）可临时使用，但限≤3天（长期使用可能反跳加重鼻塞）。
•	过敏性鼻炎：需测过敏原，口服抗组胺药（如氯雷他定）或鼻用激素喷雾（如布地奈德）。
•	慢性鼻甲问题：医生可能建议射频消融或部分切除手术。

4. 最终结论：
鼻塞可能由感冒、过敏、鼻炎等引起，建议优先尝试 盐水冲洗、蒸汽吸入和环境调整。若 症状持续超过10天、反复发作或 伴随其他严重症状（发热、黄脓涕、鼻出血），需尽快就医明确病因，避免延误慢性鼻窦炎或鼻息肉的进展。

注：上述方案综合了11条文档中高频且安全有效的建议，剔除了潜在风险方法（如直接塞葱蒜）。儿童需根据年龄选择温和方式，严重鼻塞需重视潜在疾病风险。
"""

answer_without_rag = """鼻子不通气（鼻塞）可能由多种原因引起，如感冒、过敏、鼻窦炎、鼻中隔偏曲、环境干燥或过度使用减充血剂等。以下是针对不同情况的应对措施，可分为家庭护理、药物辅助和需要就医的情况：

一、家庭护理方法
1. 蒸汽吸入
•	方法：用热水倒入大碗中，头靠近碗口，用毛巾盖住头部形成蒸汽罩，吸入热蒸汽5-10分钟。可加入几滴薄荷或桉树精油增强效果。
•	作用：缓解鼻腔干燥或充血，稀释黏液。
•	注意：避免烫伤，保持安全距离。
2. 盐水冲洗/喷雾
•	方法：
o	生理盐水冲洗：使用洗鼻壶或喷雾器，将生理盐水（比例为1升温水+1茶匙盐，煮沸冷却后使用）冲洗鼻腔，清除分泌物和过敏原。
o	鼻用盐水喷雾：药店购买现成产品，可随身携带。
•	作用：温和清洁鼻腔，保持湿润，缓解充血。
3. 热敷
•	方法：用温热（非烫）的湿毛巾敷在鼻梁和鼻窦区域（额头、颧骨、太阳穴），每次10-15分钟，每日数次。
•	作用：促进血液循环，缓解鼻窦肿胀。
4. 保持空气湿润
•	方法：使用加湿器或放置水盆在房间内，增加空气湿度（建议湿度在40%-60%），避免鼻腔干燥。
•	注意：定期清洁加湿器，防止霉菌滋生。
5. 头部抬高
•	方法：睡觉时用额外枕头将头和上半身抬高15-30度，或调整睡姿，减少鼻腔液体倒流引起的鼻塞。

二、药物缓解（按需使用）
1. 临时解充血喷雾
•	成分：羟甲唑啉（Afrin）、赛洛唑啉等。
•	使用方法：遵医嘱或说明书，连续使用不超过3天（避免药物性鼻炎）。
•	注意：孕妇、高血压患者需谨慎。
2. 抗组胺药（过敏性鼻塞）
•	适用情况：花粉、尘螨、宠物皮屑等过敏引起鼻塞。
•	药物举例：西替利嗪、氯雷他定、非索非那定（口服）；氮卓斯汀鼻喷雾（局部）。
•	注意：可能引起嗜睡，选择非镇静类成分。
3. 抗炎鼻喷剂（鼻用激素）
•	举例：氟替卡松、莫米松、布地奈德等。
•	作用：长期控制过敏或慢性鼻炎引起的鼻塞，需遵医嘱规律使用。
•	注意：起效较慢（需2-7天），但效果持久。
4. 止痛药
•	适用情况：伴有头痛或鼻窦疼痛时可口服对乙酰氨基酚或布洛芬。
5. 中药或偏方
•	举例：葱白、生姜煮水后热蒸；艾叶熏蒸；薄荷油滴润鼻。
•	注意：效果因人而异，可能存在刺激，敏感者慎用。

三、避免加重鼻塞的行为
1.	不要过度擤鼻：用力擤鼻可能导致中耳感染或鼻腔黏膜损伤，可用单侧鼻孔擤鼻法。
2.	避免鼻腔刺激物：如烟草烟雾、香水、空气清新剂等。
3.	减少过敏原接触：如花粉症患者应在花粉季节减少外出，关闭门窗，使用空气净化器。
4.	慎用减充血剂：避免长期（>7天）使用解充血鼻喷剂，可能引发反跳性充血。
5.	限制酒精和咖啡因：可能加重脱水，导致鼻腔干燥。

四、需就医的情况
如果出现以下情况，建议尽早就医：
1.	症状持续超过10天，尤其伴随发热、头痛、面部胀痛（可能为鼻窦炎）。
2.	鼻出血频繁或大量出血。
3.	呼吸困难、胸闷、皮疹等严重过敏反应。
4.	面部畸形或结构问题：如鼻中隔偏曲、鼻息肉，可能需要手术干预。
5.	儿童鼻塞持续不缓解：可能影响睡眠或进食。

五、预防措施
•	增强免疫力：均衡饮食（富含维生素C、锌）、充足睡眠、适度运动。
•	戒烟并远离二手烟：吸烟会加重鼻黏膜炎症。
•	过敏原管理：定期清洁床单（防尘螨）、去除宠物毛发、控制室内湿度（防霉菌）。

特别注意事项
•	孕妇/哺乳期女性：用药前需咨询医生，避免使用未经证实安全的药物。
•	儿童：2岁以下儿童避免使用非处方鼻用减充血剂，可咨询医生选择安全方法。

通过结合上述方法，多数鼻塞症状可缓解。若反复发作或长期不愈，建议尽早就诊耳鼻喉科，明确病因（如鼻息肉、鼻窦炎、慢性过敏等）并针对性治疗。

"""

# 定义权重参数
weights_bleu1 = (1, 0, 0, 0)
weights_bleu2 = (0.5, 0.5, 0, 0)
weights_bleu3 = (1/3, 1/3, 1/3, 0)
weights_bleu4 = (0.25, 0.25, 0.25, 0.25)

# 计算各指标的函数，传入参考答案列表和候选答案字符串
def evaluate_candidate(references, candidate):
    bleu1 = compute_bleu_n(references, candidate, weights_bleu1)
    bleu2 = compute_bleu_n(references, candidate, weights_bleu2)
    bleu3 = compute_bleu_n(references, candidate, weights_bleu3)
    bleu4 = compute_bleu_n(references, candidate, weights_bleu4)
    gleu = compute_gleu(references, candidate)
    rouge1, rouge2, rougeL = compute_rouge_scores(references, candidate)
    bert = compute_bert_score(references, candidate)
    distinct1 = compute_distinct(candidate, 1)
    distinct2 = compute_distinct(candidate, 2)
    return {
        'BLEU-1': bleu1,
        'BLEU-2': bleu2,
        'BLEU-3': bleu3,
        'BLEU-4': bleu4,
        'GLEU': gleu,
        'ROUGE-1': rouge1,
        'ROUGE-2': rouge2,
        'ROUGE-L': rougeL,
        'Distinct-1': distinct1,
        'Distinct-2': distinct2,
        'BERTScore': bert
    }

# 对两种候选答案进行评估
results_with_rag = evaluate_candidate(reference_answers, answer_with_rag)
results_without_rag = evaluate_candidate(reference_answers, answer_without_rag)

# 输出评估结果
print("=== Evaluation Results ===")
print("Metrics for candidate with RAG:")
for metric, score in results_with_rag.items():
    print(f"{metric:12s}: {score:.4f}")

print("\nMetrics for candidate without RAG:")
for metric, score in results_without_rag.items():
    print(f"{metric:12s}: {score:.4f}")


Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 2.059 seconds.
DEBUG:jieba:Loading model cost 2.059 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lowe

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


=== Evaluation Results ===
Metrics for candidate with RAG:
BLEU-1      : 0.0837
BLEU-2      : 0.0270
BLEU-3      : 0.0089
BLEU-4      : 0.0043
GLEU        : 0.0240
ROUGE-1     : 0.0520
ROUGE-2     : 0.0000
ROUGE-L     : 0.0487
Distinct-1  : 0.3853
Distinct-2  : 0.7799
BERTScore   : 0.6056

Metrics for candidate without RAG:
BLEU-1      : 0.0697
BLEU-2      : 0.0254
BLEU-3      : 0.0080
BLEU-4      : 0.0022
GLEU        : 0.0204
ROUGE-1     : 0.0192
ROUGE-2     : 0.0000
ROUGE-L     : 0.0192
Distinct-1  : 0.3781
Distinct-2  : 0.7550
BERTScore   : 0.6068
